In [8]:
import pandas as pd 
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn import preprocessing
import pylab as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [9]:
dataset_2014=pd.read_csv("Demographic and 311 calls merged - 2.csv")

In [10]:
dataset_2014.head()

,Unnamed: 0,BoroCT2010,Normalized 311 calls,Median House Value,Median Age,White only percentage,Non-white percentage,Median Rent,Median Income,Normalized cars,Normalized High school,Normalized bachelors,Normalized masters,Normalized PHD,Normalized familiy household,Normalized non-family household,Normalized owner ocuppied,Normalized renter ocuppied,Total Population,Total households
0,0,1000202,0.069812,449400.0,44.1,0.0441,0.9559,621.0,29418.0,0.092034,243.866293,157.912070,53.680224,4.939898,0.587467,0.412533,0.210416,0.789584,8308.0,3447.0
1,1,1000500,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,2,1000600,0.101374,519100.0,46.3,0.0125,0.9875,589.0,18944.0,0.068885,140.789474,115.430622,28.468900,2.751196,0.632271,0.367729,0.029778,0.970222,11354.0,3954.0
3,3,1000700,0.189928,911000.0,28.9,0.0724,0.9276,2000.0,114519.0,0.067700,15.241196,469.813554,253.625333,40.988458,0.267028,0.732972,0.201585,0.798415,8598.0,4610.0
4,4,1000800,0.118397,480600.0,45.6,0.0091,0.9909,906.0,24366.0,0.073506,158.589709,117.206643,44.922407,8.031582,0.672524,0.327476,0.017062,0.982938,9333.0,3585.0


In [11]:
len(dataset_2014)

2078

In [12]:
dataset_2014_=pd.read_csv("Demographic and 311 calls merged - 2.csv").dropna()
len(dataset_2014_)

1978

100 rows lost when dropping nan values

In [13]:
dataset_2014_.columns

Index([u'Unnamed: 0', u'BoroCT2010', u'Normalized 311 calls',
       u'Median House Value', u'Median Age', u'White only percentage',
       u'Non-white percentage', u'Median Rent', u'Median Income',
       u'Normalized cars', u'Normalized High school', u'Normalized bachelors',
       u'Normalized masters', u'Normalized PHD',
       u'Normalized familiy household', u'Normalized non-family household',
       u'Normalized owner ocuppied', u'Normalized renter ocuppied',
       u'Total Population', u'Total households'],
      dtype='object')

In [14]:
X=dataset_2014_[dataset_2014_.columns[3:18]]
Y=dataset_2014_[dataset_2014_.columns[2]]

In [16]:
X_scaled = preprocessing.scale(X)
Y_scaled = preprocessing.scale(Y)

In [17]:
#LASSO
X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X_scaled, Y_scaled, test_size=0.30, random_state=1)
X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
R2_store=[]
for i in range(-300,2):
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=i)
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_val) - Y_val) / np.var(Y_val)
    R2_store.append(R2_OS)
optim_alpha=range(-300,10)[np.where(R2_store==np.max(R2_store))[0][0]]
print "alpha = %f"%optim_alpha
Lasso = linear_model.Lasso(fit_intercept=True,alpha=optim_alpha)
Lasso.fit(X_train,Y_train)
R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
print R2_OS

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
C:\Users\juanpablo\Anaconda2\lib\site-packages\sklearn\linear_model\coordinate_descent.py:454: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  positive)
C:\Users\juanpablo\Anaconda2\lib\site-packages\sklearn\linear_model\coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


alpha = 0.000000
0.0230446614299


In [18]:
Lasso.coef_

array([ -4.02597725e-02,   7.11467636e-03,  -5.39056097e-02,
         1.39745318e-14,   3.64117140e-02,   8.66710542e-02,
         8.24742366e-02,   9.55405273e-02,   2.51427953e-02,
        -7.29322448e-02,  -3.60356051e-02,  -2.80893430e-01,
         5.96786785e-13,  -1.13054026e-01,   5.36158231e-12])

In [19]:
coefficients_result=zip(Lasso.coef_,X.columns)

In [11]:
coefficients_table=pd.DataFrame()
coefficients_table['results']=Lasso.coef_
coefficients_table['labels']=X.columns
coefficients_table.sort('results',inplace=True)
C_table=coefficients_table[abs(coefficients_table.results)>0]
C_table

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,results,labels
11,-5.198866e-01,Normalized familiy household
2,-5.091751e-01,White only percentage
13,-1.297956e-01,Normalized owner ocuppied
10,-4.892075e-04,Normalized PHD
9,-2.964694e-04,Normalized masters
0,-5.667199e-08,Median House Value
3,8.357856e-14,Non-white percentage
12,1.064437e-12,Normalized non-family household
14,6.161521e-12,Normalized renter ocuppied
5,8.871805e-07,Median Income


#Original 311-2014 file

In [20]:
complete_2014=pd.read_csv("311 - 2014 complaints with CT.csv")

C:\Users\juanpablo\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (18,41,48,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
complete_2014.head()

,Unnamed: 0,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,...,BoroCode,BoroName,CDEligibil,CT2010,CTLabel,NTACode,NTAName,PUMA,Shape_Area,Shape_Leng
0,0,29611713,12/31/2014 12:00:00 AM,06/05/2015 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,UNSANITARY CONDITION,MOLD,RESIDENTIAL BUILDING,10019.0,...,1.0,Manhattan,I,13700.0,137.0,MN17,Midtown-Midtown South,3807.0,2.435168e+06,6257.852032
1,1,29607489,12/31/2014 12:00:00 AM,01/22/2015 08:56:16 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,1-2 Family Dwelling,11205.0,...,3.0,Brooklyn,E,23300.0,233.0,BK75,Bedford,4003.0,1.849442e+06,5438.790291
2,2,29609686,12/31/2014 12:00:00 AM,12/31/2014 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10040.0,...,1.0,Manhattan,E,27700.0,277.0,MN35,Washington Heights North,3801.0,1.760156e+06,5903.151960
3,3,29608954,12/31/2014 12:00:00 AM,01/03/2015 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11373.0,...,4.0,Queens,E,46700.0,467.0,QN29,Elmhurst,4107.0,1.528978e+06,6450.862938
4,4,29609736,12/31/2014 12:00:00 AM,06/05/2015 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PAINT/PLASTER,WALL,RESIDENTIAL BUILDING,10019.0,...,1.0,Manhattan,I,13700.0,137.0,MN17,Midtown-Midtown South,3807.0,2.435168e+06,6257.852032


In [22]:
complete_2014.columns

Index([u'Unnamed: 0', u'Unique Key', u'Created Date', u'Closed Date',
       u'Agency', u'Agency Name', u'Complaint Type', u'Descriptor',
       u'Location Type', u'Incident Zip', u'Incident Address', u'Street Name',
       u'Cross Street 1', u'Cross Street 2', u'Intersection Street 1',
       u'Intersection Street 2', u'Address Type', u'City', u'Landmark',
       u'Facility Type', u'Status', u'Due Date', u'Resolution Description',
       u'Resolution Action Updated Date', u'Community Board', u'Borough',
       u'X Coordinate (State Plane)', u'Y Coordinate (State Plane)',
       u'Park Facility Name', u'Park Borough', u'School Name',
       u'School Number', u'School Region', u'School Code',
       u'School Phone Number', u'School Address', u'School City',
       u'School State', u'School Zip', u'School Not Found',
       u'School or Citywide Complaint', u'Vehicle Type',
       u'Taxi Company Borough', u'Taxi Pick Up Location',
       u'Bridge Highway Name', u'Bridge Highway Direction'

In [23]:
grouped_bytype= pd.DataFrame(complete_2014['Complaint Type'].groupby(complete_2014['BoroCT2010']).value_counts()).reset_index()
grouped_bytype.rename(columns={'level_1': 'complain_type',0: 'calls_bytype'}, inplace=True)
grouped2=grouped_bytype.groupby(grouped_bytype['BoroCT2010']).sum().reset_index()
grouped2.rename(columns={0: 'total_calls'}, inplace=True)

In [24]:
grouped_bytype.head()

,BoroCT2010,complain_type,calls_bytype
0,1000201.0,HEAT/HOT WATER,68
1,1000201.0,HEATING,22
2,1000201.0,Noise,19
3,1000201.0,ELECTRIC,14
4,1000201.0,UNSANITARY CONDITION,13


In [25]:
grouped2.columns = ['BoroCT2010', 'total_calls']

In [26]:
grouped2.head()

,BoroCT2010,total_calls
0,1000201.0,287
1,1000202.0,580
2,1000500.0,8
3,1000600.0,1151
4,1000700.0,1633


In [27]:
df2=grouped_bytype.pivot(index='BoroCT2010',columns='complain_type',values='calls_bytype')
df2.reset_index(inplace=True)
df2.head()
#df2.to_csv('2014_bytype.csv')
df3=pd.merge(df2,grouped2,on=['BoroCT2010'],how='inner')
print len(df2), len(grouped2),len(df3)
df3.head()

2162 2162 2162


complain_type,BoroCT2010,AGENCY,APPLIANCE,Adopt-A-Basket,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Asbestos,BEST/Site Safety,...,Vacant Lot,Vending,Violation of Park Rules,WATER LEAK,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment,total_calls
0,1000201.0,NaN,4.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,1.0,2.0,NaN,NaN,287
1,1000202.0,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,20.0,NaN,NaN,580
2,1000500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
3,1000600.0,NaN,9.0,NaN,9.0,1.0,NaN,NaN,1.0,1.0,...,NaN,2.0,1.0,9.0,NaN,1.0,85.0,NaN,NaN,1151
4,1000700.0,NaN,NaN,NaN,17.0,2.0,NaN,NaN,7.0,NaN,...,NaN,14.0,1.0,4.0,NaN,NaN,39.0,NaN,NaN,1633


In [28]:
df3.columns

Index([u'BoroCT2010', u'AGENCY', u'APPLIANCE', u'Adopt-A-Basket',
       u'Air Quality', u'Animal Abuse', u'Animal Facility - No Permit',
       u'Animal in a Park', u'Asbestos', u'BEST/Site Safety',
       ...
       u'Vacant Lot', u'Vending', u'Violation of Park Rules', u'WATER LEAK',
       u'Water Conservation', u'Water Quality', u'Water System',
       u'Window Guard', u'X-Ray Machine/Equipment', u'total_calls'],
      dtype='object', name=u'complain_type', length=181)

In [29]:
df3['BoroCT2010'] = (df3.BoroCT2010.astype(str).str[:-2])

In [30]:
df3['BoroCT2010'] = df3.BoroCT2010.astype(int64)

In [31]:
attributes_norm=dataset_2014_[dataset_2014_.columns[3:20]]
attributes_norm['BoroCT2010']=dataset_2014_['BoroCT2010']

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [32]:
callsbytype_attributes=pd.merge(attributes_norm,df3,on=['BoroCT2010'],how='inner')
callsbytype_attributes.head()

,Median House Value,Median Age,White only percentage,Non-white percentage,Median Rent,Median Income,Normalized cars,Normalized High school,Normalized bachelors,Normalized masters,...,Vacant Lot,Vending,Violation of Park Rules,WATER LEAK,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment,total_calls
0,449400.0,44.1,0.0441,0.9559,621.0,29418.0,0.092034,243.866293,157.912070,53.680224,...,NaN,NaN,NaN,NaN,4.0,NaN,20.0,NaN,NaN,580
1,519100.0,46.3,0.0125,0.9875,589.0,18944.0,0.068885,140.789474,115.430622,28.468900,...,NaN,2.0,1.0,9.0,NaN,1.0,85.0,NaN,NaN,1151
2,911000.0,28.9,0.0724,0.9276,2000.0,114519.0,0.067700,15.241196,469.813554,253.625333,...,NaN,14.0,1.0,4.0,NaN,NaN,39.0,NaN,NaN,1633
3,480600.0,45.6,0.0091,0.9909,906.0,24366.0,0.073506,158.589709,117.206643,44.922407,...,NaN,3.0,1.0,4.0,NaN,1.0,33.0,NaN,NaN,1105
4,1000000.0,33.3,0.0799,0.9201,2000.0,170455.0,0.211176,56.273063,331.180812,235.239852,...,NaN,8.0,2.0,1.0,2.0,NaN,39.0,NaN,NaN,1103


In [33]:
len(callsbytype_attributes)

1978

In [34]:
regressors=attributes_norm.columns[0:20]
attributes_norm.columns

Index([u'Median House Value', u'Median Age', u'White only percentage',
       u'Non-white percentage', u'Median Rent', u'Median Income',
       u'Normalized cars', u'Normalized High school', u'Normalized bachelors',
       u'Normalized masters', u'Normalized PHD',
       u'Normalized familiy household', u'Normalized non-family household',
       u'Normalized owner ocuppied', u'Normalized renter ocuppied',
       u'Total Population', u'Total households', u'BoroCT2010'],
      dtype='object')

In [35]:
regressors

Index([u'Median House Value', u'Median Age', u'White only percentage',
       u'Non-white percentage', u'Median Rent', u'Median Income',
       u'Normalized cars', u'Normalized High school', u'Normalized bachelors',
       u'Normalized masters', u'Normalized PHD',
       u'Normalized familiy household', u'Normalized non-family household',
       u'Normalized owner ocuppied', u'Normalized renter ocuppied',
       u'Total Population', u'Total households', u'BoroCT2010'],
      dtype='object')

In [36]:
#attributes
types_of_calls=df3.columns[1:180]
df3.columns

Index([u'BoroCT2010', u'AGENCY', u'APPLIANCE', u'Adopt-A-Basket',
       u'Air Quality', u'Animal Abuse', u'Animal Facility - No Permit',
       u'Animal in a Park', u'Asbestos', u'BEST/Site Safety',
       ...
       u'Vacant Lot', u'Vending', u'Violation of Park Rules', u'WATER LEAK',
       u'Water Conservation', u'Water Quality', u'Water System',
       u'Window Guard', u'X-Ray Machine/Equipment', u'total_calls'],
      dtype='object', name=u'complain_type', length=181)

##Lasso Regression for each type

###first, the code for just one type, then I will write the code for every type of complain

In [37]:
types_of_calls[20]

'Bus Stop Shelter Placement'

In [38]:
typeof=types_of_calls[20]

In [39]:
A2=np.append(np.append(np.append(regressors,typeof),'BoroCT2010'),'total_calls')
myframe1=callsbytype_attributes[A2].dropna()

In [40]:
print len(myframe1)
myframe1.head()

161


,Median House Value,Median Age,White only percentage,Non-white percentage,Median Rent,Median Income,Normalized cars,Normalized High school,Normalized bachelors,Normalized masters,...,Normalized familiy household,Normalized non-family household,Normalized owner ocuppied,Normalized renter ocuppied,Total Population,Total households,BoroCT2010,Bus Stop Shelter Placement,BoroCT2010,total_calls
4,1000000.0,33.3,0.0799,0.9201,2000.0,170455.0,0.211176,56.273063,331.180812,235.239852,...,0.517028,0.482972,0.135704,0.864296,1396.0,646.0,1000900,1.0,1000900,1103
5,516400.0,42.7,0.0769,0.9231,1419.0,83259.0,0.242118,165.853659,376.422764,195.934959,...,0.483176,0.516824,0.735277,0.264723,1681.0,743.0,1001001,1.0,1001001,147
16,866100.0,33.1,0.0611,0.9389,2000.0,76250.0,0.089295,69.508197,540.327869,125.245902,...,0.301217,0.698783,0.089833,0.910167,1944.0,986.0,1002202,1.0,1002202,547
20,480700.0,38.2,0.0415,0.9585,1032.0,34612.0,0.137147,125.315543,282.185359,84.926073,...,0.449191,0.550809,0.086423,0.913577,7409.0,3277.0,1002800,1.0,1002800,1289
33,617000.0,29.9,0.0795,0.9205,1859.0,91235.0,0.040971,56.538692,517.696461,223.905219,...,0.205764,0.794236,0.085024,0.914976,9543.0,4233.0,1004000,1.0,1004000,1866


In [41]:
myframe1.columns

Index([u'Median House Value', u'Median Age', u'White only percentage',
       u'Non-white percentage', u'Median Rent', u'Median Income',
       u'Normalized cars', u'Normalized High school', u'Normalized bachelors',
       u'Normalized masters', u'Normalized PHD',
       u'Normalized familiy household', u'Normalized non-family household',
       u'Normalized owner ocuppied', u'Normalized renter ocuppied',
       u'Total Population', u'Total households', u'BoroCT2010',
       u'Bus Stop Shelter Placement', u'BoroCT2010', u'total_calls'],
      dtype='object')

In [42]:
X=myframe1[['Median House Value', 'Median Age', 'White only percentage',
       'Median Rent', 'Median Income',
       'Normalized cars', 'Normalized High school', 'Normalized bachelors',
       'Normalized masters', 'Normalized PHD',
       'Normalized familiy household', 'Normalized owner ocuppied', 
       'Total Population']]
Y=myframe1[typeof]/myframe1['Total households']

In [44]:
X_scaled = preprocessing.scale(X)
Y_scaled = preprocessing.scale(Y)

In [50]:
#LASSO
X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X_scaled, Y_scaled, test_size=0.35, random_state=1)
X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.35, random_state=1)
R2_store=[]
for i in range(-40,40):
    Lasso = linear_model.Lasso(fit_intercept=True,alpha=i)
    Lasso.fit(X_train,Y_train)
    R2_OS = 1 - np.var(Lasso.predict(X_val) - Y_val) / np.var(Y_val)
    R2_store.append(R2_OS)
optim_alpha=range(-40,40)[np.where(R2_store==np.max(R2_store))[0][0]]
print "alpha = %f"%optim_alpha
Lasso = linear_model.Lasso(fit_intercept=True,alpha=optim_alpha)
Lasso.fit(X_train,Y_train)
R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
print R2_OS

alpha = 0.000000
0.122482758188


C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


###Now, for every type:

In [52]:
results={}
for typeof in types_of_calls:
    A2=np.append(np.append(np.append(regressors,typeof),'BoroCT2010'),'total_calls')
    myframe1=callsbytype_attributes[A2].dropna()
    if len(myframe1)>100:
        results[typeof]={}
        X=myframe1[['Median House Value', 'Median Age', 'White only percentage',
       'Median Rent', 'Median Income',
       'Normalized cars', 'Normalized High school', 'Normalized bachelors',
       'Normalized masters', 'Normalized PHD',
       'Normalized familiy household', 'Normalized owner ocuppied', 
       'Total Population']]
        Y=myframe1[typeof]/myframe1['Total households']
        X_scaled = preprocessing.scale(X)
        Y_scaled = preprocessing.scale(Y)
        #LASSO
        X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X_scaled, Y_scaled, test_size=0.30, random_state=1)
        X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
        R2_store=[]
        for i in range(-40,40):
            Lasso = linear_model.Lasso(fit_intercept=True,alpha=i)
            Lasso.fit(X_train,Y_train)
            R2_OS = 1 - np.var(Lasso.predict(X_val) - Y_val) / np.var(Y_val)
            R2_store.append(R2_OS)
        optim_alpha=range(-40,40)[np.where(R2_store==np.max(R2_store))[0][0]]
        results[typeof]['best_alpha']=optim_alpha
        Lasso = linear_model.Lasso(fit_intercept=True,alpha=optim_alpha)
        Lasso.fit(X_train,Y_train)
        R2_OS = 1 - np.var(Lasso.predict(X_test) - Y_test) / np.var(Y_test)
        results[typeof]['best_R2']=R2_OS
        results[typeof]['sample']=len(myframe1)

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:22: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:28: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


In [53]:
results_frame = pd.DataFrame(index=range(0,len(results.keys())), columns=['type','best_alpha','R2','sample_size'])

In [54]:
results_frame['type']=results.keys()
results_frame.head()
for i in results_frame.index:
    typeof=results_frame.loc[i,'type']
    results_frame.iloc[i,1]=results[typeof]['best_alpha']
    results_frame.iloc[i,2]=results[typeof]['best_R2']  
    results_frame.iloc[i,3]=results[typeof]['sample'] 

In [55]:
results_frame.sort('R2',ascending=False)

C:\Users\juanpablo\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,type,best_alpha,R2,sample_size
98,Overgrown Tree/Branches,0,0.432464,1794
84,Root/Sewer/Sidewalk Condition,0,0.397758,1536
112,Damaged Tree,0,0.382449,1868
58,Drinking Water,0,0.378444,114
19,PLUMBING,0,0.366198,1806
27,Urinating in Public,0,0.363969,312
125,Missed Collection (All Materials),0,0.345552,1856
24,Sweeping/Inadequate,0,0.32483,299
2,WATER LEAK,0,0.318514,1677
83,DOOR/WINDOW,0,0.314638,1631


In [40]:
results_frame.to_csv('Lasso_initial_results V3.csv')